## Libraries

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
#print(os.listdir("../input"))
#print(os.listdir("../input/glove-global-vectors-for-word-representation"))
#print(os.listdir("../input/jigsaw-unintended-bias-in-toxicity-classification"))

# Any results you write to the current directory are saved as output.

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import datetime
import os
import pandas as pd
import numpy as np
import pkg_resources
import seaborn as sns
import time
import scipy.stats as stats

from sklearn import metrics
from sklearn import model_selection

from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding
from keras.layers import Input
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Dense
from keras.optimizers import RMSprop, Adadelta , Nadam
from keras.models import Model
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


## load data from older competition (for oversampling)

In [3]:
old_train = pd.read_csv('../input/cleaned-toxic-comments/train_preprocessed.csv', encoding='UTF-8', index_col=0)
print('loaded %d train records' % len(old_train))

old_test = pd.read_csv('../input/cleaned-toxic-comments/test_preprocessed.csv', encoding='UTF-8', index_col=0)
print('loaded %d train records' % len(old_test))


test_labels = pd.read_csv('../input/bandida/test_labels.csv', encoding='UTF-8', index_col=0)
print('loaded %d train records' % len(test_labels))

loaded 159571 train records
loaded 153164 train records
loaded 153164 train records


In [4]:
old_train=old_train.reset_index().set_index('id')
old_train=old_train[['comment_text','toxic']]
old_train.head()

,comment_text,toxic
id,,
0000997932d777bf,explanation why the edits made under my userna...,0.0
000103f0d9cfb60f,d aww he matches this background colour i m s...,0.0
000113f07ec002fd,hey man i m really not trying to edit war it...,0.0
0001b41b1c6bb37e,more i can t make any real suggestions on im...,0.0
0001d958c54c6e35,you sir are my hero any chance you remember...,0.0


In [5]:
old_test=old_test.reset_index().set_index('id')
old_test=old_test[['comment_text']]
old_test.head()

,comment_text
id,
00001cee341fdb12,yo bitch ja rule is more succesful then you ll...
0000247867823ef7,from rfc the title is fine as it is imo
00013b17ad220c46,sources zawe ashton on lapland
00017563c3f7919a,if you have a look back at the source the in...
00017695ad8997eb,i don t anonymously edit articles at all


In [6]:
old_not_toxic_test = pd.concat([old_test,test_labels],axis=1)
print(old_not_toxic_test.shape)
old_not_toxic_test.head()

(153164, 7)


,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
id,,,,,,,
00001cee341fdb12,yo bitch ja rule is more succesful then you ll...,-1,-1,-1,-1,-1,-1
0000247867823ef7,from rfc the title is fine as it is imo,-1,-1,-1,-1,-1,-1
00013b17ad220c46,sources zawe ashton on lapland,-1,-1,-1,-1,-1,-1
00017563c3f7919a,if you have a look back at the source the in...,-1,-1,-1,-1,-1,-1
00017695ad8997eb,i don t anonymously edit articles at all,-1,-1,-1,-1,-1,-1


In [7]:
# agregar reset_index

In [8]:
old_train.head()

,comment_text,toxic
id,,
0000997932d777bf,explanation why the edits made under my userna...,0.0
000103f0d9cfb60f,d aww he matches this background colour i m s...,0.0
000113f07ec002fd,hey man i m really not trying to edit war it...,0.0
0001b41b1c6bb37e,more i can t make any real suggestions on im...,0.0
0001d958c54c6e35,you sir are my hero any chance you remember...,0.0


In [9]:
old_not_toxic=old_train[old_train['toxic']==0]
old_not_toxic.head()

,comment_text,toxic
id,,
0000997932d777bf,explanation why the edits made under my userna...,0.0
000103f0d9cfb60f,d aww he matches this background colour i m s...,0.0
000113f07ec002fd,hey man i m really not trying to edit war it...,0.0
0001b41b1c6bb37e,more i can t make any real suggestions on im...,0.0
0001d958c54c6e35,you sir are my hero any chance you remember...,0.0


In [10]:
old_not_toxic_test=old_not_toxic_test[old_not_toxic_test['toxic']==0]
old_not_toxic_test=old_not_toxic_test[['comment_text','toxic']]
old_not_toxic_test.head()

,comment_text,toxic
id,,
0001ea8717f6de06,thank you for understanding i think very high...,0
000247e83dcc1211,dear god this site is horrible,0
0002f87b16116a7f,somebody will invariably try to add religion ...,0
0003e1cccfd5a40a,it says it right there that it is a type the...,0
00059ace3e3e9a53,before adding a new product to the list make...,0


## load data from bias competition

In [11]:
train = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/train.csv', encoding='UTF-8', index_col=0)
print('loaded %d train records' % len(train))
test = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/test.csv', encoding='UTF-8', index_col=0)
print('loaded %d test records' % len(test))

/opt/conda/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


loaded 1804874 train records
loaded 97320 test records


In [12]:
test.head()

,comment_text
id,
7000000,Jeff Sessions is another one of Trump's Orwell...
7000001,I actually inspected the infrastructure on Gra...
7000002,No it won't . That's just wishful thinking on ...
7000003,Instead of wringing our hands and nibbling the...
7000004,how many of you commenters have garbage piled ...


## pre-process the data set

In [13]:
#train = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/train.csv')
print('loaded %d records' % len(train))

# Make sure all comment_text values are strings
train['comment_text'] = train['comment_text'].astype(str) 
test['comment_text'] = test['comment_text'].astype(str) 

# List all identities
identity_columns = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness','asian','transgender','latino','atheist']

#identity_columns=id

# Convert target and identity columns to booleans
def convert_to_bool(df, col_name):
    df[col_name] = np.where(df[col_name] >= 0.5, True, False)
    
def convert_dataframe_to_bool(df):
    bool_df = df.copy()
    for col in ['target'] + identity_columns: #
        convert_to_bool(bool_df, col)
    return bool_df

def convert_dataframe_to_bool_test(df):
    bool_df = df.copy()
    for col in ['comment_text']: #
        convert_to_bool(bool_df, col)
    return bool_df

train = convert_dataframe_to_bool(train)
#test = convert_dataframe_to_bool_test(test)

loaded 1804874 records


In [14]:
#NEW
print(old_not_toxic.shape)
g1=old_not_toxic.iloc[0:24082,]
g2=old_not_toxic.iloc[24082:24082+15011,]
g3=old_not_toxic.iloc[24082+15011:24082+15011+33568,]
g4=old_not_toxic.iloc[24082+15011+33568:24082+15011+33568+18836,]
g5=old_not_toxic.iloc[24082+15011+33568+18836:24082+15011+33568+18836+3694,]
g6=old_not_toxic.iloc[24082+15011+33568+18836+3694:24082+15011+33568+18836+3694+2682,]
g7=old_not_toxic.iloc[24082+15011+33568+18836+3694+2682:24082+15011+33568+18836+3694+2682+11224,]
g8=old_not_toxic.iloc[24082+15011+33568+18836+3694+2682+11224:24082+15011+33568+18836+3694+2682+11224+7496,]
g9=old_not_toxic_test.iloc[0:1934,]
g10=old_not_toxic_test.iloc[1934:1934+1229,]
g11=old_not_toxic_test.iloc[1934+1229:1934+1229+139,]
g12=old_not_toxic_test.iloc[1934+1229+139:1934+1229+139+130,]


g1=g1.copy(deep=True)
g2=g2.copy(deep=True)
g3=g3.copy(deep=True)
g4=g4.copy(deep=True)
g5=g5.copy(deep=True)
g6=g6.copy(deep=True)
g7=g7.copy(deep=True)
g8=g8.copy(deep=True)
g9=g9.copy(deep=True)
g10=g10.copy(deep=True)
g11=g11.copy(deep=True)
g12=g12.copy(deep=True)


g1['black']=1
g2['homosexual_gay_or_lesbian']=1
g3['white']=1
g4['muslim']=1
g5['psychiatric_or_mental_illness']=1
g6['jewish']=1
g7['male']=1
g8['female']=1
g9['transgender']=1
g10['latino']=1
g11['asian']=1
g12['atheist']=1

print('black',g1.shape)
print('homosexual_gay_or_lesbian',g2.shape)
print('white',g3.shape)
print('muslim',g4.shape)
print('psychiatric_or_mental_illness',g5.shape)
print('jewish',g6.shape)
print('male',g7.shape)
print('female',g8.shape)
print('transgender',g9.shape)
print('latino',g10.shape)
print('asian',g11.shape)
print('atheist',g12.shape)

print('train before append:',train.shape)
train=pd.concat([train,g1,g2,g3,g4,g5,g6,g7,g8,g9,g10,g11,g12])
train = convert_dataframe_to_bool(train)
print('train after append:',train.shape)

g1.head()

(144277, 2)
black (24082, 3)
homosexual_gay_or_lesbian (15011, 3)
white (33568, 3)
muslim (18836, 3)
psychiatric_or_mental_illness (3694, 3)
jewish (2682, 3)
male (11224, 3)
female (7496, 3)
transgender (1934, 3)
latino (1229, 3)
asian (139, 3)
atheist (130, 3)
train before append: (1804874, 44)


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:58: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



train after append: (1924899, 45)


,comment_text,toxic,black
id,,,
0000997932d777bf,explanation why the edits made under my userna...,0.0,1
000103f0d9cfb60f,d aww he matches this background colour i m s...,0.0,1
000113f07ec002fd,hey man i m really not trying to edit war it...,0.0,1
0001b41b1c6bb37e,more i can t make any real suggestions on im...,0.0,1
0001d958c54c6e35,you sir are my hero any chance you remember...,0.0,1


In [15]:
cant = pd.DataFrame([
                    [train[train['male']==True].target.value_counts()[0],
                     train[train['male']==True].target.value_counts()[1]],
    [train[train['female']==True].target.value_counts()[0],
                     train[train['female']==True].target.value_counts()[1]],
    [train[train['homosexual_gay_or_lesbian']==True].target.value_counts()[0],
                     train[train['homosexual_gay_or_lesbian']==True].target.value_counts()[1]],
    [train[train['christian']==True].target.value_counts()[0],
                     train[train['christian']==True].target.value_counts()[1]],
    [train[train['jewish']==True].target.value_counts()[0],
                     train[train['jewish']==True].target.value_counts()[1]],
    [train[train['muslim']==True].target.value_counts()[0],
                     train[train['muslim']==True].target.value_counts()[1]],
    [train[train['black']==True].target.value_counts()[0],
                     train[train['black']==True].target.value_counts()[1]],
    [train[train['white']==True].target.value_counts()[0],
                     train[train['white']==True].target.value_counts()[1]],
    [train[train['psychiatric_or_mental_illness']==True].target.value_counts()[0],
                     train[train['psychiatric_or_mental_illness']==True].target.value_counts()[1]],
    [train[train['asian']==True].target.value_counts()[0],
                     train[train['asian']==True].target.value_counts()[1]],
    [train[train['transgender']==True].target.value_counts()[0],
                     train[train['transgender']==True].target.value_counts()[1]],
    [train[train['latino']==True].target.value_counts()[0],
                     train[train['latino']==True].target.value_counts()[1]],
    [train[train['atheist']==True].target.value_counts()[0],
                     train[train['atheist']==True].target.value_counts()[1]]    
      ]
                    ,index= identity_columns, columns=['False','True'])

cant['Total']=cant['False']+cant['True']
cant['pFalse']=cant['False']/cant['Total']
cant.sort_values(by='pFalse',inplace=True)
cant

,False,True,Total,pFalse
latino,2845,388,3233,0.879988
transgender,3901,532,4433,0.879991
psychiatric_or_mental_illness,7553,1030,8583,0.879995
jewish,9093,1240,10333,0.879996
homosexual_gay_or_lesbian,22887,3121,26008,0.879998
black,34305,4678,38983,0.879999
male,49023,6685,55708,0.879999
female,53614,7311,60925,0.880000
white,51612,7038,58650,0.880000
muslim,35061,4781,39842,0.880001


## Split the data into 80% train and 20% validate sets

In [16]:
train_df, validate_df = model_selection.train_test_split(train, test_size=0.1)
print('%d train comments, %d validate comments' % (len(train_df), len(validate_df)))


1732409 train comments, 192490 validate comments


## Create a text tokenizer

In [17]:
MAX_NUM_WORDS = 10000
TOXICITY_COLUMN = 'target'
TEXT_COLUMN = 'comment_text'

# Create a text tokenizer.
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(train_df[TEXT_COLUMN])

# All comments must be truncated or padded to be the same length.
MAX_SEQUENCE_LENGTH = 250
def pad_text(texts, tokenizer):
    return pad_sequences(tokenizer.texts_to_sequences(texts), maxlen=MAX_SEQUENCE_LENGTH)

## Define and train a Convolutional Neural Net for classifying toxic comments

In [18]:
from keras import backend as K 
K.clear_session()

In [19]:
EMBEDDINGS_PATH = "../input/bandida/glove.6B.100d.txt"
EMBEDDINGS_DIMENSION = 100
DROPOUT_RATE = 0.5
LEARNING_RATE = 0.00001
NUM_EPOCHS = 5
BATCH_SIZE = 200

opt1=RMSprop(lr=LEARNING_RATE)
opt2=Adadelta()
opt3=Nadam()
OPT=opt3



def train_model(train_df, validate_df, tokenizer):
    # Prepare data
    train_text = pad_text(train_df[TEXT_COLUMN], tokenizer)
    validate_text = pad_text(validate_df[TEXT_COLUMN], tokenizer)
    
    #train_labels = train_df[TOXICITY_COLUMN]
    #validate_labels = validate_df[TOXICITY_COLUMN]
    
    train_labels = to_categorical(train_df[TOXICITY_COLUMN])
    validate_labels = to_categorical(validate_df[TOXICITY_COLUMN])

    # Load embeddings
    print('loading embeddings')
    embeddings_index = {}
    with open(EMBEDDINGS_PATH) as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs

    embedding_matrix = np.zeros((len(tokenizer.word_index) + 1,
                                 EMBEDDINGS_DIMENSION))
    num_words_in_embedding = 0
    for word, i in tokenizer.word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            num_words_in_embedding += 1
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector

    # Create model layers.
    def get_convolutional_neural_net_layers():
        """Returns (input_layer, output_layer)"""
        sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
        embedding_layer = Embedding(len(tokenizer.word_index) + 1,
                                    EMBEDDINGS_DIMENSION,
                                    weights=[embedding_matrix],
                                    input_length=MAX_SEQUENCE_LENGTH,
                                    trainable=False)
        x = embedding_layer(sequence_input)
        x = Conv1D(128, 2, activation='relu', padding='same')(x)
        x = MaxPooling1D(5, padding='same')(x)
        x = Conv1D(128, 3, activation='relu', padding='same')(x)
        x = MaxPooling1D(5, padding='same')(x)
        x = Conv1D(128, 4, activation='relu', padding='same')(x)
        x = MaxPooling1D(40, padding='same')(x)
        x = Flatten()(x)
        x = Dropout(DROPOUT_RATE)(x)
        x = Dense(128, activation='relu')(x)
        preds = Dense(2, activation='softmax')(x)
        return sequence_input, preds

    
    # Compile model.
    print('compiling model')
    input_layer, output_layer = get_convolutional_neural_net_layers()
    model = Model(input_layer, output_layer)
    
    #model.load_weights('../input/weights/my_model_weights-2.h5')
    
    model.compile(loss='categorical_crossentropy',
                  optimizer=OPT,
                  metrics=['acc'])

    fBestModel = 'best_model.h5'
    best_model = ModelCheckpoint(fBestModel, verbose=0, save_best_only=True)
    # Train model.
    print('training model')
    model.fit(train_text,
              train_labels,
              batch_size=BATCH_SIZE,
              epochs=NUM_EPOCHS,
              validation_data=(validate_text, validate_labels),
              verbose=2,
              callbacks=[best_model])

    return model

model = train_model(train_df, validate_df, tokenizer)
#model.save_weights('my_model_weights.h5')

Instructions for updating:
Colocations handled automatically by placer.
loading embeddings
compiling model
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
training model
Instructions for updating:
Use tf.cast instead.
Train on 1732409 samples, validate on 192490 samples
Epoch 1/5
 - 85s - loss: 0.1507 - acc: 0.9463 - val_loss: 0.1457 - val_acc: 0.9430
Epoch 2/5
 - 82s - loss: 0.1396 - acc: 0.9492 - val_loss: 0.1435 - val_acc: 0.9478
Epoch 3/5
 - 82s - loss: 0.1368 - acc: 0.9500 - val_loss: 0.1332 - val_acc: 0.9515
Epoch 4/5
 - 82s - loss: 0.1353 - acc: 0.9503 - val_loss: 0.1330 - val_acc: 0.9517
Epoch 5/5
 - 81s - loss: 0.1342 - acc: 0.9509 - val_loss: 0.1370 - val_acc: 0.9494


## Generate model predictions on the validation set

In [20]:
MODEL_NAME = 'my_model'
validate_df[MODEL_NAME] = model.predict(pad_text(validate_df[TEXT_COLUMN], tokenizer))[:, 1]

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


## Define bias metrics, then evaluate our new model for bias using the validation set predictions

In [21]:
SUBGROUP_AUC = 'subgroup_auc'
BPSN_AUC = 'bpsn_auc'  # stands for background positive, subgroup negative
BNSP_AUC = 'bnsp_auc'  # stands for background negative, subgroup positive

def compute_auc(y_true, y_pred):
    try:
        return metrics.roc_auc_score(y_true, y_pred)
    except ValueError:
        return np.nan

def compute_subgroup_auc(df, subgroup, label, model_name):
    subgroup_examples = df[df[subgroup]]
    return compute_auc(subgroup_examples[label], subgroup_examples[model_name])

def compute_bpsn_auc(df, subgroup, label, model_name):
    """Computes the AUC of the within-subgroup negative examples and the background positive examples."""
    subgroup_negative_examples = df[df[subgroup] & ~df[label]]
    non_subgroup_positive_examples = df[~df[subgroup] & df[label]]
    examples = subgroup_negative_examples.append(non_subgroup_positive_examples)
    return compute_auc(examples[label], examples[model_name])

def compute_bnsp_auc(df, subgroup, label, model_name):
    """Computes the AUC of the within-subgroup positive examples and the background negative examples."""
    subgroup_positive_examples = df[df[subgroup] & df[label]]
    non_subgroup_negative_examples = df[~df[subgroup] & ~df[label]]
    examples = subgroup_positive_examples.append(non_subgroup_negative_examples)
    return compute_auc(examples[label], examples[model_name])

def compute_bias_metrics_for_model(dataset,
                                   subgroups,
                                   model,
                                   label_col,
                                   include_asegs=False):
    """Computes per-subgroup metrics for all subgroups and one model."""
    records = []
    for subgroup in subgroups:
        record = {
            'subgroup': subgroup,
            'subgroup_size': len(dataset[dataset[subgroup]])
        }
        record[SUBGROUP_AUC] = compute_subgroup_auc(dataset, subgroup, label_col, model)
        record[BPSN_AUC] = compute_bpsn_auc(dataset, subgroup, label_col, model)
        record[BNSP_AUC] = compute_bnsp_auc(dataset, subgroup, label_col, model)
        records.append(record)
    return pd.DataFrame(records).sort_values('subgroup_auc', ascending=True)

bias_metrics_df = compute_bias_metrics_for_model(validate_df, identity_columns, MODEL_NAME, TOXICITY_COLUMN)
bias_metrics_df


,bnsp_auc,bpsn_auc,subgroup,subgroup_auc,subgroup_size
12,0.879839,0.915535,atheist,0.849268,147
4,0.920783,0.901146,jewish,0.863557,1024
10,0.929049,0.894541,transgender,0.871982,463
9,0.920196,0.909289,asian,0.883195,501
1,0.947308,0.897702,female,0.900377,6087
3,0.926397,0.920758,christian,0.901271,4054
0,0.950699,0.895315,male,0.901991,5534
5,0.946373,0.907410,muslim,0.907041,4016
11,0.950506,0.908756,latino,0.915429,339
7,0.963323,0.909086,white,0.925949,5985


## Calculate the final score

In [22]:
def calculate_overall_auc(df, model_name):
    true_labels = df[TOXICITY_COLUMN]
    predicted_labels = df[model_name]
    return metrics.roc_auc_score(true_labels, predicted_labels)

def power_mean(series, p):
    total = sum(np.power(series, p))
    return np.power(total / len(series), 1 / p)

def get_final_metric(bias_df, overall_auc, POWER=-5, OVERALL_MODEL_WEIGHT=0.25):
    bias_score = np.average([
        power_mean(bias_df[SUBGROUP_AUC], POWER),
        power_mean(bias_df[BPSN_AUC], POWER),
        power_mean(bias_df[BNSP_AUC], POWER)
    ])
    return (OVERALL_MODEL_WEIGHT * overall_auc) + ((1 - OVERALL_MODEL_WEIGHT) * bias_score)
    
get_final_metric(bias_metrics_df, calculate_overall_auc(validate_df, MODEL_NAME))

0.9212648312778482

## Prediction on Test data

In [23]:
#test = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/test.csv')
submission = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/sample_submission.csv', index_col='id')

In [24]:
submission['prediction'] = model.predict(pad_text(test[TEXT_COLUMN], tokenizer))[:, 1]
submission.to_csv('submission.csv')